In [3]:
import os
import pickle
import pandas as pd

def preprocess_value(value):
    return 0.5 + ((value + 26225) / (2 * 26225) * 9)


for pnum in range(1,11):
    ann_path = os.getcwd() + '/annotations/sub' + str(pnum) + '_joystick.txt'
    ann_file = pd.read_csv(ann_path, sep='\t', header=None)
    ann_file.columns = ['time', 'axis1', 'axis2']
    # ann_selected = ann_file.loc[(ann_file['axis1'] != 0) & (ann_file['axis2'] != 0), :]

    # ann_selected['axis1'] = ann_selected.iloc[:,1].apply(preprocess_value)
    # ann_selected['axis2'] = ann_selected.iloc[:,2].apply(preprocess_value)

    
    phy_path = os.getcwd() + '/physiological/sub' + str(pnum) + '_DAQ.txt'
    phy_file = pd.read_csv(phy_path, sep='\t', header=None)
    phy_file.columns = ['time', 'ecg', 'bvp', 'gsr', 'rsp', 'temp', 'emg_zygo', 'emg_coru', 'emg_trap']

    start_index = None
    end_index = None

    consecutive_ranges_a = []
    for index, row in ann_file.iterrows():
        if not (row['axis1'] == 0 and row['axis2'] == 0):
            if start_index is None:
                start_index = index
            end_index = index
        else:
            if start_index is not None:
                consecutive_ranges_a.append((start_index, end_index))
                start_index = None
                end_index = None

    if start_index is not None:
        consecutive_ranges_a.append((start_index, end_index))

    mask_to_get = pd.Series(False, index=ann_file.index)
    for start, end in consecutive_ranges_a:
        mask_to_get[start:end] = True

    ann_selected = ann_file[mask_to_get]

    consecutive_ranges_p = []
    for start, end in consecutive_ranges_a:
        closest_index_s = (phy_file['time'] - ann_file['time'][start]).abs().idxmin()
        closest_index_e = (phy_file['time'] - ann_file['time'][end]).abs().idxmin()
        consecutive_ranges_p.append((closest_index_s, closest_index_e))
    
    mask_to_get = pd.Series(False, index=phy_file.index)
    for start, end in consecutive_ranges_p:
        mask_to_get[start:end] = True

    phy_selected = phy_file[mask_to_get]

    dict_data = {'signal':
            {'ECG': phy_selected.iloc[:,1].to_numpy(),
            'BVP': phy_selected.iloc[:,2].to_numpy(),
            'GSR': phy_selected.iloc[:,3].to_numpy(),
            'RSP': phy_selected.iloc[:,4].to_numpy(),
            'TEMP': phy_selected.iloc[:,5].to_numpy(),
            'emg_zygo': phy_selected.iloc[:,6].to_numpy(),
            'emg_coru': phy_selected.iloc[:,7].to_numpy(),
            'emg_trap': phy_selected.iloc[:,8].to_numpy()},
            'label':
            {'AROUSAL': ann_selected[['axis1']].to_numpy(),
            'VALENCE': ann_selected[['axis2']].to_numpy()},
            'subject': f"S{pnum}"}

    folder_name = f"S{pnum}"
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)

    pkl_file_path = os.path.join(folder_name, f"S{pnum}.pkl")
    with open(pkl_file_path, 'wb') as pkl_file:
        pickle.dump(dict_data, pkl_file)